# HSC Re-Run: Making Forced Photometry Light Curves from Scratch

<br>Owners: **Justin Myles** ([@jtmyles](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@jtmyles)), **Phil Marshall** ([@drphilmarshall](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@drphilmarshall))
<br>Last Verified to Run: **2018-08-10**
<br>Verified Stack Release: **16.0**

This project addresses issue [#63: HSC Re-run](https://github.com/LSSTScienceCollaborations/StackClub/issues/63)

This notebook demonstrates the pipeline described in the [LSST Science Piplines data processing tutorial](https://pipelines.lsst.io/), from ingesting images (using the [obs_subaru](https://github.com/lsst/obs_subaru) package) through image processing, coaddition, source detection and object measurement all the way through to measuring forced photometry light curves in a small patch of the HSC sky (in the [ci_hsc](https://github.com/lsst/ci_hsc/) repository). It does this by calling a `bash` script, having first identified a minimal data set for demonstration purposes.  

### Learning Objectives:
After working through and studying this notebook you should be able to understand how to use the DRP pipeline from image visualization through to a forced photometry light curve. Specific learning objectives include: 
   1. [Configuring](https://pipelines.lsst.io/v/w-2018-12/modules/lsst.pipe.base/command-line-task-config-howto.html) and executing pipeline command line tasks.
   2. The sequence of steps involved in the DRP pipeline.
   
Other techniques that are demonstrated, but not emphasized, in this notebook are
   1. Using the `butler` to fetch data
   2. Visualizing data with the LSST Stack

### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.


## Set Up

In [ ]:
import os
# os.system("eups list lsst_distrib") #todo
import sys
import matplotlib.pyplot as plt
%matplotlib inline
# import eups.setupcmd

## A Minimal Test Dataset
We'll use the `ci_hsc` test dataset, and reprocess just enough of it to make a 5-point i-band light curve. This means choosing a single sky patch, and finding just the visits that lie within it. Jim Bosch shows how to do this in [this community.lsst.org post](https://community.lsst.org/t/visualizing-source-images-in-a-coadd/441/2), given a coadd image. Our problem is that the `ci_hsc` repo doesn't contain any coadds, until we make them...  So perhaps we need to work from the HSC dataset instead? or run the script, then interrogate the coadds it makes, then go back and 


## Pipeline Preview

The pipeline described in the [LSST Science Piplines data processing tutorial](https://pipelines.lsst.io/) contains a complete set of command line tasks that can be assembled into an end-to-end data reduction pipeline script. Let's see what this script looks like.

In [ ]:
! cat Re-RunHSC.sh

We'll come back to each step in turn throughout the rest of this notebook.

## Part I: Interacting with data. Introduction to the Butler
https://pipelines.lsst.io/getting-started/data-setup.html

Part I runs the following command-line tasks
```
eups list lsst_distrib
setup -j -r /home/jmyles/repositories/ci_hsc
echo "lsst.obs.hsc.HscMapper" > /home/jmyles/DATA/_mapper
ingestImages.py /home/jmyles/DATA /home/jmyles/repositories/ci_hsc/raw/*.fits --mode=link
ln -s /home/jmyles/repositories/ci_hsc/CALIB/ /home/jmyles/DATA/CALIB
installTransmissionCurves.py /home/jmyles/DATA
mkdir -p /home/jmyles/DATA/ref_cats
ln -s /home/jmyles/repositories/ci_hsc/ps1_pv3_3pi_20170110 /home/jmyles/DATA/ref_cats/ps1_pv3_3pi_20170110
```

In [ ]:
HOME = os.environ['HOME']
DATAREPO = "{}/repositories/ci_hsc/".format(HOME)
DATADIR = "{}/DATA/".format(HOME)
os.system("mkdir -p {}".format(DATADIR));

In [ ]:
#!setup -j -r /home/jmyles/repositories/ci_hsc

setup = eups.setupcmd.EupsSetup(["-j","-r", DATAREPO])
status = setup.run()
print('setup exited with status {}'.format(status))

A Butler needs a *mapper* file "to find and organize data in a format specific to each camera." We write this file to the data repository so that any instantiated Butler object knows which mapper to use.

In [ ]:
with open(DATADIR + "_mapper", "w") as f:
    f.write("lsst.obs.hsc.HscMapper")

In [ ]:
# ingest script
!ingestImages.py DATADIR /home/jmyles/repositories/ci_hsc/raw/*.fits --mode=link

In [ ]:
#!installTransmissionCurves.py /home/jmyles/DATA

from lsst.obs.hsc import makeTransmissionCurves, HscMapper
from lsst.daf.persistence import Butler

butler = Butler(outputs={'root': datadir, 'mode': 'rw', 'mapper': HscMapper})

for start, nested in makeTransmissionCurves.getFilterTransmission().items():
    for name, curve in nested.items():
        if curve is not None:
            butler.put(curve, "transmission_filter", filter=name)
for start, nested in makeTransmissionCurves.getSensorTransmission().items():
    for ccd, curve in nested.items():
        if curve is not None:
            butler.put(curve, "transmission_sensor", ccd=ccd)
for start, curve in makeTransmissionCurves.getOpticsTransmission().items():
    if curve is not None:
        butler.put(curve, "transmission_optics")
for start, curve in makeTransmissionCurves.getAtmosphereTransmission().items():
    if curve is not None:
        butler.put(curve, "transmission_atmosphere")

In [ ]:
# ingest calibration images into Butler repo
os.system("ln -s {} {}".format(datarepo + "CALIB/", datadir + "CALIB"))

# ingest reference catalog into Butler repo
os.system("mkdir -p {}".format(DATADIR + "ref_cats"))
os.system("ln -s {}ps1_pv3_3pi_20170110 {}ref_cats/ps1_pv3_3pi_20170110".format(DATAREPO, DATADIR))

## Part 2: Calibrating single frames
https://pipelines.lsst.io/getting-started/processccd.html

In [ ]:
"""
!processCcd.py DATA --rerun processCcdOutputs --id
# all cl tasks write output datasets to a Butler repo
# --rerun configured to write to processCcdOutputs
# other option is --output
"""

In [ ]:
!which processCcd.py
"""
processCcd.py
from lsst.pipe.tasks.processCcd import ProcessCcdTask

ProcessCcdTask.parseAndRun()
"""

In [ ]:
# show source of lsst.pipe.tasks.processCcd
# emacs /opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/pipe_tasks/16.0+1/python/lsst/pipe/tasks/processCcd.py


In [ ]:
from lsst.pipe.tasks.processCcd import ProcessCcdTask, ProcessCcdConfig
processCcdTaskInstance = ProcessCcdTask(butler=butler)

In [ ]:
from stackclub import where_is
where_is(processCcdTaskInstance, in_the="source")

In [ ]:
processCcdConfig = ProcessCcdConfig()

In [ ]:
"""
# review what data will be processed
!processCcd.py DATA --rerun processCcdOutputs --id --show data
# id allows you to select data by data ID
# unspecified id selects all raw data
# example IDs: raw, filter, visit, ccd, field
# show data turns on dry-run mode
"""

In [ ]:
#!which processCcd.py

# Part 3: Displaying exposures and source tables output by processCcd.py
https://pipelines.lsst.io/getting-started/display.html

This part of the tutorial is omitted for now.

# Part 4: Coadding images
https://pipelines.lsst.io/getting-started/coaddition.html

* A sky map is a tiling of the celestial sphere. It is composed of one or more tracts.
* A tract is composed of one or more overlapping patches. Each tract has a WCS.

In [ ]:
"""# make a discrete sky map that covers the exposures that have already been processed
!makeDiscreteSkyMap.py DATA --id --rerun processCcdOutputs:coadd --config skyMap.projection="TAN"

# the configuration field specifies the WCS Projection
# one of the FITS WCS projection codes, such as:
#           - STG: stereographic projection
#           - MOL: Molleweide's projection
#           - TAN: tangent-plane projection
"""

# Part 6: Multi-band catalog analysis
https://pipelines.lsst.io/getting-started/multiband-analysis.html

In [ ]:
import os
import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler(inputs=DATADIR + 'rerun/coaddForcedPhot/')

Access the sources identified from the coadd images

In [ ]:
rSources = butler.get('deepCoadd_forced_src', {'filter': 'HSC-R', 'tract': 0, 'patch': '1,1'})
iSources = butler.get('deepCoadd_forced_src', {'filter': 'HSC-I', 'tract': 0, 'patch': '1,1'})

Throw out negative fluxes, and convert fluxes to magnitudes.

In [ ]:
rCoaddCalib = butler.get('deepCoadd_calexp_calib',  {'filter': 'HSC-R', 'tract': 0, 'patch': '1,1'})
iCoaddCalib = butler.get('deepCoadd_calexp_calib',  {'filter': 'HSC-I', 'tract': 0, 'patch': '1,1'})

rCoaddCalib.setThrowOnNegativeFlux(False)
iCoaddCalib.setThrowOnNegativeFlux(False)

rMags = rCoaddCalib.getMagnitude(rSources['base_PsfFlux_flux'])
iMags = iCoaddCalib.getMagnitude(iSources['base_PsfFlux_flux'])

Select stars from catalog

In [ ]:
deblended = rSources['deblend_nChild'] == 0

refTable = butler.get('deepCoadd_ref', {'filter': 'HSC-R^HSC-I', 'tract': 0, 'patch': '1,1'})
inInnerRegions = refTable['detect_isPatchInner'] & refTable['detect_isTractInner'] # define inner regions
isSkyObject = refTable['merge_peak_sky'] # reject sky objects
isPrimary = refTable['detect_isPrimary']

isStellar = iSources['base_ClassificationExtendedness_value'] < 1.
isGoodFlux = ~iSources['base_PsfFlux_flag']
selected = isPrimary & isStellar & isGoodFlux

Make color-magnitude diagram.

In [ ]:
plt.style.use('seaborn-notebook')
plt.figure(1, figsize=(4, 4), dpi=140)
plt.scatter(rMags[selected] - iMags[selected],
                        iMags[selected],
                        edgecolors='None', s=2, c='k')
plt.xlim(-0.5, 3)
plt.ylim(25, 14)
plt.xlabel('$r-i$')
plt.ylabel('$i$')
plt.subplots_adjust(left=0.125, bottom=0.1)
plt.show()